# Running Prompt Functions Inline


Import Semantic Kernel SDK from pypi.org

In [2]:
# Note: if using a virtual environment, do not run this cell
%pip install -U semantic-kernel
from semantic_kernel import __version__

__version__

Note: you may need to restart the kernel to use updated packages.


'1.18.2'

Initial configuration for the notebook to run properly.

In [3]:
# Make sure paths are correct for the imports

import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)


sys.path.append(grandparent_dir)

### Configuring the Kernel

Let's get started with the necessary configuration to run Semantic Kernel. For Notebooks, we require a `.env` file with the proper settings for the model you use. Create a new file named `.env` and place it in this directory. Copy the contents of the `.env.example` file from this directory and paste it into the `.env` file that you just created.

**NOTE: Please make sure to include `GLOBAL_LLM_SERVICE` set to either OpenAI, AzureOpenAI, or HuggingFace in your .env file. If this setting is not included, the Service will default to AzureOpenAI.**

#### Option 1: using OpenAI

Add your [OpenAI Key](https://openai.com/product/) key to your `.env` file (org Id only if you have multiple orgs):

```
GLOBAL_LLM_SERVICE="OpenAI"
OPENAI_API_KEY="sk-..."
OPENAI_ORG_ID=""
OPENAI_CHAT_MODEL_ID=""
OPENAI_TEXT_MODEL_ID=""
OPENAI_EMBEDDING_MODEL_ID=""
```
The names should match the names used in the `.env` file, as shown above.

#### Option 2: using Azure OpenAI

Add your [Azure OpenAI Service key](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio) settings to the `.env` file in the same folder:

```
GLOBAL_LLM_SERVICE="AzureOpenAI"
AZURE_OPENAI_API_KEY="..."
AZURE_OPENAI_ENDPOINT="https://..."
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME="..."
AZURE_OPENAI_TEXT_DEPLOYMENT_NAME="..."
AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME="..."
AZURE_OPENAI_API_VERSION="..."
```
The names should match the names used in the `.env` file, as shown above.

For more advanced configuration, please follow the steps outlined in the [setup guide](./CONFIGURING_THE_KERNEL.md).

The [previous notebook](./02-running-prompts-from-file.ipynb)
showed how to define a semantic function using a prompt template stored on a file.

In this notebook, we'll show how to use the Semantic Kernel to define functions inline with your python code. This can be useful in a few scenarios:

- Dynamically generating the prompt using complex rules at runtime
- Writing prompts by editing Python code instead of TXT files.
- Easily creating demos, like this document

Prompt templates are defined using the SK template language, which allows to reference variables and functions. Read [this doc](https://aka.ms/sk/howto/configurefunction) to learn more about the design decisions for prompt templating.

For now we'll use only the `{{$input}}` variable, and see more complex templates later.

Almost all semantic function prompts have a reference to `{{$input}}`, which is the default way
a user can import content from the context variables.


Prepare a semantic kernel instance first, loading also the AI service settings defined in the [Setup notebook](00-getting-started.ipynb):


Let's define our kernel for this example.

In [4]:
from semantic_kernel.kernel import Kernel

kernel = Kernel()

In [5]:
from services import Service

from samples.service_settings import ServiceSettings

service_settings = ServiceSettings.create()

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = (
    Service.AzureOpenAI
    if service_settings.global_llm_service is None
    else Service(service_settings.global_llm_service.lower())
)
print(f"Using service type: {selectedService}")

Using service type: Service.AzureOpenAI


We now configure our Chat Completion service on the kernel.

In [6]:
# Remove all services so that this cell can be re-run without restarting the kernel
kernel.remove_all_services()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"
    kernel.add_service(
        OpenAIChatCompletion(
            service_id=service_id,
        ),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
        ),
    )

Let's use a prompt to create a semantic function used to summarize content, allowing for some creativity and a sufficient number of tokens.

The function will take in input the text to summarize.


In [7]:
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings, OpenAIChatPromptExecutionSettings
from semantic_kernel.prompt_template import InputVariable, PromptTemplateConfig

prompt = """{{$input}}
Summarize the content above.
"""

if selectedService == Service.OpenAI:
    execution_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo",
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = AzureChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-35-turbo",
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="summarize",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The user input", is_required=True),
    ],
    execution_settings=execution_settings,
)

summarize = kernel.add_function(
    function_name="summarizeFunc",
    plugin_name="summarizePlugin",
    prompt_template_config=prompt_template_config,
)

Set up some content to summarize, here's a book review of the recent book by Ruchir Sharma called "What went wrong with Capitalism". The review was generated by AI.


In [12]:
input_text = """
Ruchir Sharma’s “What Went Wrong with Capitalism” offers a compelling critique of modern economic systems, delving into the intricate relationship between government intervention and market dynamics. Sharma’s analysis is both historical and contemporary, providing readers with a nuanced understanding of how capitalism has evolved and where it has faltered.
The book begins by challenging the widely held belief in a golden age of small government. Sharma argues that this notion is largely a myth, pointing out that even during the Reagan and Thatcher eras, government influence continued to grow. This sets the stage for a broader discussion on the expanding role of government since the 1930s, highlighting how government spending has significantly increased over the decades.
One of the central themes of the book is the impact of easy money policies. Sharma critiques central bank interventions and low interest rates, which he believes have distorted capitalism by encouraging excessive risk-taking and creating a culture of debt. This, in turn, has led to a bailout culture, where the expectation of government bailouts during crises has fostered moral hazard in financial decision-making.
Sharma also explores the historical evolution of American economic thought, drawing on the differing views of figures like Alexander Hamilton and Thomas Jefferson. This historical perspective enriches the reader’s understanding of the ideological foundations of American capitalism and how these ideas have shaped contemporary economic policies.
The book is particularly critical of the permanent adoption of Keynesian deficit spending, which was originally intended as a temporary measure during crises. Sharma warns that this approach has led to rising government deficits and an impending global economic crisis. He argues that the continuous rise of big government poses significant risks to economic stability and growth.
In addition to fiscal policies, Sharma examines the increase in government regulations across industries, which he believes has further expanded the state’s role in the economy. He also touches on the growth of defense and intelligence agencies, contributing to what he describes as the national security state.
Overall, “What Went Wrong with Capitalism” is a thought-provoking and insightful analysis of the challenges facing modern capitalism. Sharma’s critique is well-researched and balanced, offering readers a comprehensive overview of the complex interplay between government intervention and market forces. Whether you are a student of economics, a policy maker, or simply someone interested in understanding the current economic landscape, this book provides valuable insights into the factors that have shaped and continue to influence capitalism today.
"""

...and run the summary function:


In [13]:
summary = await kernel.invoke(summarize, input=input_text)

print(summary)

Ruchir Sharma's book, "What Went Wrong with Capitalism," offers a critical examination of modern economic systems, focusing on the interplay between government intervention and market dynamics. Sharma debunks the myth of a golden age of small government, noting that government influence has grown since the 1930s, even during the Reagan and Thatcher eras. A central theme is the impact of easy money policies and central bank interventions, which Sharma argues have encouraged excessive risk-taking and created a culture of debt, leading to a bailout culture and moral hazard. He critiques the permanent adoption of Keynesian deficit spending, warning that it has led to rising government deficits and potential economic crises. Sharma also discusses the increase in government regulations and the expansion of defense and intelligence agencies, contributing to a national security state. The book provides a historical perspective on American economic thought and offers a comprehensive analysis of

# Using ChatCompletion for Semantic Plugins


You can also use chat completion models (like `gpt-35-turbo` and `gpt4`) for creating plugins. Normally you would have to tweak the API to accommodate for a system and user role, but SK abstracts that away for you by using `kernel.add_service` and `AzureChatCompletion` or `OpenAIChatCompletion`


Here's one more example of how to write an inline Semantic Function that gives a TLDR for a piece of text using a ChatCompletion model


In [10]:
kernel.remove_all_services()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"
    kernel.add_service(
        OpenAIChatCompletion(
            service_id=service_id,
        ),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
        ),
    )

In [11]:
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings, OpenAIChatPromptExecutionSettings

prompt = """
{{$input}}

Give me the TLDR in 5 words or less.
"""

text = """
    1) A robot may not injure a human being or, through inaction,
    allow a human being to come to harm.

    2) A robot must obey orders given it by human beings except where
    such orders would conflict with the First Law.

    3) A robot must protect its own existence as long as such protection
    does not conflict with the First or Second Law.
"""

if selectedService == Service.OpenAI:
    execution_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo",
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = AzureChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-35-turbo",
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="tldr",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The user input", is_required=True),
    ],
    execution_settings=execution_settings,
)

tldr_function = kernel.add_function(
    function_name="tldrFunction",
    plugin_name="tldrPlugin",
    prompt_template_config=prompt_template_config,
)

summary = await kernel.invoke(tldr_function, input=text)

print(f"Output: {summary}")

Output: Robots prioritize human safety first.
